In [1]:
import requests
import pandas as pd

In [2]:
# Key Bea = 30f6c00168msh1b42af1260e2970p1cb797jsn775f4f1bf53e
# Key Bea2 = 7da7d1d9f2mshd70f16dedff8827p11e4cejsnc105329a0d9c
# Key Ceci = dab5a0167dmsh4ab1543f0fd8e0ap1d9b21jsnd66b36d85213
# Key Carla = f600d33197mshe21c869c19877bbp16c957jsn8dea9e68b50b

def api_call(url):
    headers = {
        "x-rapidapi-key": "TU API", 
        "x-rapidapi-host": "moviesdatabase.p.rapidapi.com"
    }

    call = requests.get(url, headers=headers)
    print(f"The API call gave us a response of type: {call.status_code}")
    if call.status_code != 200:
        print(f"The reason for the failure is: {call.reason}")
        return None
    else:
        return call.json()

def movies_by_genre(genre_list):
    movies_list = []

    for genre in genre_list:
        for i in range(1, 2): # 2, 3 / 3, 4 / ...
            url_movies = f'https://moviesdatabase.p.rapidapi.com/titles?genre={genre}&startYear=2010&titleType=movie&page={i}&endYear=2024&limit=50'
            response_dict_movies = api_call(url_movies)

            if response_dict_movies and "results" in response_dict_movies:
                for result in response_dict_movies["results"]:
                    result["genre"] = genre
                movies_list.extend(response_dict_movies["results"])

            print(f"\n ITERACION {genre} {i} \n")
            if response_dict_movies:
                print(f"\nThe answer for movies is type: {type(response_dict_movies)}, the length is: {len(response_dict_movies)} and the keys are: {response_dict_movies.keys()}\n")
                print(response_dict_movies)

    return movies_list

def shorts_by_genre(genre_list):
    shorts_list = []

    for genre in genre_list:
        for i in range(1, 2): 
            url_short = f'https://moviesdatabase.p.rapidapi.com/titles?genre={genre}&startYear=2010&titleType=short&page={i}&endYear=2024&limit=50'
            response_dict_shorts = api_call(url_short)

            if response_dict_shorts and "results" in response_dict_shorts:
                for result in response_dict_shorts["results"]:
                    result["genre"] = genre
                shorts_list.extend(response_dict_shorts["results"])

            print(f"\n ITERACION {genre} {i} \n")
            if response_dict_shorts:
                print(f"\nThe answer for shorts is type: {type(response_dict_shorts)}, the length is: {len(response_dict_shorts)} and the keys are: {response_dict_shorts.keys()}\n")
                print(response_dict_shorts)

    return shorts_list

def process_results(results_list):
    full_list = []
    for item in results_list:
        item_data = []
        item_data.append(item["titleType"].get("text"))
        item_data.append(item["titleText"].get("text"))
        item_data.append(item["releaseYear"].get("year"))
        try:
            item_data.append(item["releaseDate"].get("month"))
        except:
            item_data.append("none")
        item_data.append(item["id"])
        item_data.append(item["genre"])
        
        full_list.append(tuple(item_data))
    return full_list

def combine_results(genre_list):
    movies_list = movies_by_genre(genre_list)
    shorts_list = shorts_by_genre(genre_list)
    
    combined_list = movies_list + shorts_list
    processed_list = process_results(combined_list)
    
    return processed_list

# call to main function
genre_list = ["Drama"] # Genres: "Action", "Drama", "Comedy"
combined_results = combine_results(genre_list)

# Create DataFrame
list_headers = ["type", "name", "release_year", "release_month", "id_imdb", "genre"]
df_full_movie_list = pd.DataFrame(combined_results, columns=list_headers)

# Set 'id_imdb' as index
df_full_movie_list.set_index('id_imdb', inplace=True)

# Show DataFrame
df_full_movie_list


The API call gave us a response of type: 200

 ITERACION Drama 1 


The answer for movies is type: <class 'dict'>, the length is: 4 and the keys are: dict_keys(['page', 'next', 'entries', 'results'])

{'page': '1', 'next': '/titles?titleType=movie&genre=Drama&page=2&limit=50&startYear=2010&endYear=2024', 'entries': 50, 'results': [{'_id': '61e58ea278c012aa34446beb', 'id': 'tt10579986', 'primaryImage': {'id': 'rm878544897', 'width': 1217, 'height': 1792, 'url': 'https://m.media-amazon.com/images/M/MV5BNDFmYWRhYmMtMDBiNS00OWZkLTk3ZmYtODMyZjdlMDFjNWNlXkEyXkFqcGdeQXVyNjA1NjIzMzQ@._V1_.jpg', 'caption': {'plainText': "Mom's Coming (2024)", '__typename': 'Markdown'}, '__typename': 'Image'}, 'titleType': {'displayableProperty': {'value': {'plainText': '', '__typename': 'Markdown'}, '__typename': 'DisplayableTitleTypeProperty'}, 'text': 'Movie', 'id': 'movie', 'isSeries': False, 'isEpisode': False, 'categories': [{'value': 'movie', '__typename': 'TitleTypeCategory'}], 'canHaveEpisodes': False, 

,type,name,release_year,release_month,genre
id_imdb,,,,,
tt10579986,Movie,Mom's Coming,2024,11,Drama
tt10597228,Movie,Love Song and Power,2024,1,Drama
tt11097384,Movie,Spaceman,2024,3,Drama
tt11152168,Movie,IF,2024,5,Drama
tt11822244,Movie,Manje Bistre 3,2024,7,Drama
...,...,...,...,...,...
tt27827381,Short,Frank,2024,none,Drama
tt27832399,Short,It Is What It Is,2024,4,Drama
tt27833184,Short,Fissures,2024,6,Drama


In [3]:
df_full_movie_list.to_csv("data/api/list_drama_range_1_2_API.csv")